In [68]:
# this code is used for creating features having the coordinatal data cleaned as its input to be used to prediction

In [ ]:
import numpy as np
from scipy.fftpack import fft
import csv
import pandas
import scipy
import math
import sys
import time
import copy
import matplotlib
import json
import os
from scipy import interpolate
from scipy import signal
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import pandas
import os.path
import json 
import scipy.stats as ss
import numpy as np
import pylab as plt


In [69]:
# corresponding codes for different body joints based on json files
[RHIP, RANK, RSHO, RKNE, RWRI, RELB, REAR] = [8,10,2,9,4,3,16]
[LHIP, LANK, LSHO, LKNE, LWRI, LELB, LEAR] = [11,13,5,12,7,6,17]
NOSE = 0
NECK = 1
topoint = lambda x: range(2*x,2*x+2)

alldata = pandas.read_csv("../alldata.csv")
videos = pandas.read_csv("../video_list.csv")
alldata = alldata[alldata["side"] == "R"]
video = videos["Video_File"].apply(lambda x: x.__str__()[37:-4])
exam = videos["Exam_ID"]
map_ev = dict(zip(exam,video))
alldata["videoid"] = alldata["examid"].apply(lambda x: map_ev[int(x)] if int(x) in map_ev.keys() else None)
cadence = alldata["cadence"]
alldata.to_csv('out.csv')
alldata = pandas.read_csv("./out.csv")

D:\Users\soes\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (300) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
# remove the elements that are nan and come consecutively at the end
def removeEndingNans(dat):
    a = np.asarray(dat)
    while (1):
        if np.isnan(a[-1]):
            a = np.delete(a, a.size-1)
        else:
            break
    return a;

In [71]:
# fill the nan elements by interpolation
def fill_nan(A):
    #interpolate to fill nan values
    A = np.asarray(A);
    inds = np.arange(A.shape[0])
    cond = np.logical_and(np.isfinite(A),A!=0)
    good = [i for i, x in enumerate(cond) if x]
    f = interpolate.interp1d(inds[good], A[good],kind="linear",fill_value="extrapolate")
    B = np.where(cond,A,f(inds))
    #print (np.isnan(B))
    return B

In [73]:
# Fast Fourier Transform 
def fastFT( data ):
    # Number of sample points
    Fs = 24
    T = 1/Fs
    L = len(data)
    t = np.arange(L)*T
    x = data
    x = fill_nan(data)
    y = fft(x)
    p2 = np.abs(y/L)
    f = Fs*np.arange(L/2)/L
    p1 = 2*p2[0:f.size]
    f[10:] = np.apply_along_axis(lambda x: scipy.ndimage.filters.gaussian_filter1d(x,1),arr=f[10:],axis=0)
    return f[10:], p1[10:]

In [1]:
# numerical intergation
def numInteg(x,y):
    # test of numerical integration
    yy = (y*y)*x
    numIntegValue = np.trapz(yy,x)

    return numIntegValue

In [75]:
# doing preprocessing on the frames such as 
# -normalizing the coordinates
# -centralizing the body coordinates

def preprocessTheView(view2):
    ref_frame = 150

    res = view2
    num_parts = 18
    processed_cols = [True,True,False] * (num_parts)

    res = res[:,processed_cols]
    M = np.nanmean(res,axis=0)
    M = M[topoint(RHIP)].tolist() * num_parts
    centered = res - M
    # distance between the right hip and right ankle
    for i in (4,5,16,17,20,21):
        res[:,i] = fill_nan(res[:,i])
        res[:,i] = np.apply_along_axis(lambda x: scipy.ndimage.filters.gaussian_filter1d(x,1),arr=res[:,i],axis=0)
    
    scale_scalar_ank = np.linalg.norm(res[ref_frame,topoint(RHIP)] - res[ref_frame,topoint(RANK)])
    scale_scalar_sho = np.linalg.norm(res[ref_frame,topoint(RHIP)] - res[ref_frame,topoint(RSHO)])
    scale = scale_scalar_sho+scale_scalar_ank)/2
    
    centered = centered / scale
    return centered

In [76]:
# calculate the average of steplengths
def averageStepLength(dxRANK,dxLANK):
    aveStepLen = np.mean(abs(dxRANK-dxLANK))
    return aveStepLen

In [77]:
# calculate the average of speeds measured in 10 time windows
def averageSpeed(dxNECK):
    sumOfSpeeds = 0
    for x in range(10, 110, 10):
        sumOfSpeeds = sumOfSpeeds + abs(dxNECK[24+x]-dxNECK[x])
    aveSpeed = sumOfSpeeds / 10
    return aveSpeed

In [78]:
# creating features by preprocessing the frames, doing fft, doing numerical integation
def createFeatures():
    #removes if csv already exists then writes first row as the name of the features
    os.remove('features.csv')
    features_id = np.array(['r1','r2','r3','r4','r5','r6','r7','r8','r9','r10','r11','r12','l1','l2','l3','l4','l5','l6','l7','l8','l9','l10','l11','l12',
                           'aveStepLength','aveSpeed','actualCad','actualStepLen','actualSpeed','folder'])
    with open(r'features.csv', 'a',newline='') as f:
        writer = csv.writer(f)
        writer.writerow(features_id)
    dir='./data'
    list = []
    freq = [0,1,2,3,4,5,6,7,8,9,10,11,12]
    feat = np.zeros(26)
    output = np.zeros(30)
    counter = 0
    for folder in os.listdir(dir):
        folder_path= os.path.join(dir,folder)
        if os.path.isdir(folder_path) and len([name for name in os.listdir(folder_path)if name.endswith(".npy")])!=0:
            counter = counter + 1
            list = [folder] + list
            view2_path= os.path.join(dir,folder,'view2.npy')
            view2 = np.load(view2_path)

            centered = preprocessTheView(view2)

            dxRANK = centered[:,2*RANK]
            dxLANK = centered[:,2*LANK]
            dxNECK = centered[:,2*NECK]

            dxRANK = removeEndingNans(dxRANK)
            dxLANK = removeEndingNans(dxLANK)
            dxNECK = removeEndingNans(dxNECK)

            dxRANK = fill_nan(dxRANK)
            dxLANK = fill_nan(dxLANK)
            dxNECK = fill_nan(dxNECK)

            aveSpeed = averageSpeed(dxNECK)

            fRANK, ampRANK = fastFT( dxRANK )
            fLANK, ampLANK = fastFT( dxLANK ) 
            fRANK = fill_nan(fRANK)
            fLANK = fill_nan(fLANK)
            ampRANK = fill_nan(ampRANK)
            ampLANK = fill_nan(ampLANK)

            feat[0] = numInteg(fRANK[fRANK<=freq[1]],ampRANK[fRANK<=freq[1]])
            for i in range(1, 12):
                feat[i] = numInteg(fRANK[np.where(np.logical_and(fRANK>=freq[i], fRANK<=freq[i+1]))],ampRANK[np.where(np.logical_and(fRANK>=freq[i], fRANK<=freq[i+1]))])

            feat[12] = numInteg(fLANK[fLANK<=freq[1]],ampLANK[fLANK<=freq[1]])
            for i in range(13, 24):
                feat[i] = numInteg(fLANK[np.where(np.logical_and(fLANK>=freq[i-12], fLANK<=freq[i+1-12]))],ampLANK[np.where(np.logical_and(fLANK>=freq[i-12], fLANK<=freq[i+1-12]))])

            feat[24] = averageStepLength(dxRANK,dxLANK)
            feat[25] = aveSpeed
            output[0:26] = feat[:]

            fff = int(folder)
            fff = fff
            cad = (alldata.loc[alldata["videoid"] == fff]["cadence"]) 
            stepLen = (alldata.loc[alldata["videoid"] == fff]["steplen"]) 
            speed = (alldata.loc[alldata["videoid"] == fff]["speed"]) 
            if  cad.empty == False:
                output[26] = cad.iloc[0] # cadence of that row
                output[27] = stepLen.iloc[0] # stepLength of that row
                output[28] = speed.iloc[0] # speed of that row
                output[29] = folder
                with open(r'features.csv', 'a',newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow(output) 
            if folder == 12803901: 
                print (folder)
                plt.figure()
                plt.plot(fRANK, ampRANK)
                plt.show()
